# <오늘 할 것: blog 앱 만들기 >

# 1. 기본 틀 만들기

# 2. 글 조회 페이지 만들기
- 2.1 base 템플릿 분리하기

# 3. 로그인 폼 만들기
- 3.1 class로 View 만들기
- **3.2 상대경로/절대경로/url rendering**
- 3.3 로그인 유저이름 띄우기
- **부록) {% %}와 {{}}의 차이**

# 4. 포스트 작성 폼 만들기
- 4.1 새로운 글 작성하기
- 4.2 글 수정하기
    - 4.2.1 **Form을 쓰면 좋은점**

## 1. 기본 틀 만들기

$ `python manage.py startapp blog`로 앱 생성하기

`settings.py`의 `INSTALLED_APPS` 변수 에 `blog` 추가하기

`blog/models.py`에 `POST` 테이블을 클래스로 정의하기

In [ ]:
from django.db import models
from django.utils import timezone

# Create your models here.

class Post(models.Model):
    author = models.ForeignKey('auth.User', on_delete=models.CASCADE)   #auth.User를 지울 때 해당 User가 작성한 Post도 지우도록 설정
    title = models.CharField(max_length=200)
    text = models.TextField()  # 글자수에 제한 없는 텍스트
    created_date = models.DateTimeField(
        default=timezone.now)  # 날짜와 시간
    published_date = models.DateTimeField(
        blank=True, null=True) #  필드가 폼에서 빈 채로 저장되는 것을 허용; blank는 어플리케이션 측면, null은 데이터베이스 측면에서의 빈값을 의미

    def publish(self):
        self.published_date = timezone.now()
        self.save()

    def __str__(self):
        return self.title

`blog/admin.py`에 `POST` 테이블 등록하기

$ `python manage.py makemigrations` & `python manage.py migrate`로 DB에 반영하기

SQLite로 실제로 DB에 저장된 결과를 확인하면 `blog/models.py`에 정의했던 것과 조금 다르다는 것을 확인할 수 있다.

<img src=blog.jpg>

정의한 적이 없는 `author_id` 칼럼이 있다.

--> django에서 자동적으로 `_id`를 붙여서 생성한 열이다.

SQLite에서 `데이터베이스 구조` 탭에서 `blog_post`의 스키마를 직접 보면 다음과 같다.

In [ ]:
CREATE TABLE "blog_post" 
("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, 
 "title" varchar(200) NOT NULL, 
 "text" text NOT NULL, 
 "created_date" datetime NOT NULL, 
 "published_date" datetime NULL, 
 "author_id" integer NOT NULL REFERENCES 
 "auth_user" ("id") DEFERRABLE INITIALLY DEFERRED);

`author_id`라는 칼럼은 `auth_user`라는 테이블의 `id`칼럼을 참조(`REFERENCES`)하여 생성되었음을 알 수 있다. 원본 칼럼이 `AUTOINCREMENT` 설정이 되어있기 때문에 그걸 참조해서 생성된 `author_id` 역시 `AUTOINCREMENT` 설정을 따른다.

# 2. 글 조회 페이지 만들기
## 2.1 base 템플릿 분리하기

페이지에 상관없이 지속적으로 노출되어야 하는 요소들이 있다면, 코드의 효율성을 위해 **base template**을 별도로 분리하면 좋다.

e.g. 블로그 제목, 페이지 하단의 copyright, 주소 등

~~base template은 템플릿의 템플릿이라 하겠다~~

- `base.html`에는 다른 소스로 채워질 공간을 마킹하고
- 끼워넣을 템플릿에는 1. 어떤 base에 끼워넣을지 선언하고, 2. 끼워넣을 부분을 마킹해준다.

먼저 `base.html`부터 보자.
- `{% block content %}`
- `{% endblock %}`

로 다른 소스로 채워질 공간을 마킹한다

In [ ]:
<!--베이스 템플릿-->
<font color="red"><h1> My blog </h1></font>

{{username}} 님 로그인하셨습니다. <br><br><br>

{% block content %}    
{% endblock %}

<br><br><br>
copyright....<br>
서울특별시...

아래는 끼워넣을 템플릿이다.
- `{% extends 'blog/base.html' %}`

로 어떤 베이스 템플릿에 끼워넣을 건지를 명시하고
- `{% block content %}`
- `{% endblock %}`

로 끼워넣을 부분을 감싸준다.

In [ ]:
<!--끼워넣을 템플릿-->
{% extends 'blog/base.html' %}

{% block content %}

{% for d in data %}
<a href="{% url 'detail' d.pk %}"> {{d.title}} </a> <br>
{% endfor %}

{% endblock %}

이걸 브라우저에서 띄워서 보면 다음과 같이 보인다.

<img src=basetemp.jpg>

글 상세 내용을 보는 `detail.html`도 `base.html`을 참조했기 때문에, 글 제목을 클릭해서 들어가도 여전히 상/하단의 내용들이 고정되어 나타나는 것을 확인할 수 있다.

<img src=detail.jpg>

# 3. 로그인 폼 만들기

## 3.1 class로 View 만들기

요청을 get/post로 나누어서 처리하고 싶을 때, 이때까지는 if문으로 처리했지만, 두 경우를 그냥 class로 묶을 수도 있다. 이게 더 편리하고 코드도 더 직관적이고 깔끔하다.

- `views.py`에서 할 것
- `urls.py`에서 할 것

으로 나누어 볼 수 있다.

먼저, `views.py`에서 할 것은 class를 정의해주는 것이다.

이때 get과 post를 각각 함수로 만들어준다.

- 보통 `get`은 `render`로 form을 띄우고
- `post`는 `redirect`로 끝난다.

In [ ]:
class LoginView(View):

    def get(self, request):                          
        return render(request, 'blog/login.html')    

    def post(self, request):                        
        username = request.POST.get('username')     
        password = request.POST.get('password')
        user = authenticate(username=username, password=password)
        if user == None:
            return redirect('login')   #여기 쓴 건 경로가 아니라, urls에서 정의한 경로의 name이다.
        request.session['username'] = username   #세션 정보 생성 및 유저이름 저장
        return redirect('list')

`urls.py`에서는 해당 class를 View로 바꿔주어 경로에 연결시켜준다.

`.as_view()` 메소드를 쓰면 class를 view로 호출할 수 있다.

In [ ]:
from django.urls import path
from . import views  #현재 경로에서 views를 불러오고

urlpatterns = [
    ...
    path('login/', views.LoginView.as_view(), name='login'),
    #views.py에 정의된 클래스인 LoginView를 as_view 메소드로 View로 바꾼다!
    ...
]

## 3.2 상대경로/절대경로/url rendering

`blog/list.html` 참조

#### 1. 상대경로는 current 기준이다
- 아무것도 쓰지 않거나, `./`으로 시작한다.
    

In [ ]:
{% for d in data %}
<a href="{{d.pk}}/detail"> {{d.title}} </a>
{% endfor %}

#### 2. 절대경로는 base directory(root) 기준이다.
- `/`으로 시작한다.
    

In [ ]:
{% for d in data %}
<a href="/blog/{{d.pk}}/detail"> {{d.title}} </a>
{% endfor %}

#### 3. url rendering은 `urls.py`에 명시한 경로의 `name`으로 호출한다.
- `{% url 'url-name' param1, param2, param3 %}` 형태로 쓴다.
 

In [ ]:
{% for d in data %}
<a href="{% url "detail" d.pk %}"> {{d.title}} </a>
{% endfor %}

### 중요: view를 html로 만들어줄 때는 3을 쓴다.
- 왜냐면 상대경로는 전달받은 경로를 current 뒤에 붙여버리기 때문에 난감해질 때가 많다.
- 그리고 절대경로는 안 쓸 수 있다면 안 쓰는 게 맞다(바꾸기 너무 귀찮음)

## 3.3 로그인 유저 이름 띄우기

세션정보에서 유저명을 읽어와서 로그인 유저 이름을 페이지에 띄운다. 페이지 변화에 무관하게 지속적으로 노출시키기 위해 `base.html` 상단에 포함시킬 거다.

먼저 `views.py`에서 다음과 같이 정의해준다.

In [ ]:
def list(request):
    username = request.session['username']
    user = User.objects.get(username=username)
    data = Post.objects.all().filter(author=user)
    context = {"data": data, "username": username}
    return render(request, "blog/list.html", context)

def detail(request, pk):
    p = get_object_or_404(Post, pk=pk)
    username = request.session['username']
    context = {"d": p, "username": username}
    return render(request, 'blog/detail.html', context)

유저 정보를 띄워야 할 템플릿인 `list.html`와 `detail.html`이 호출될 때 전달되는 데이터에 세션정보에서 읽어온 `username`을 포함시켰다.

그리고 이걸 이제 두 템플릿의 템플릿인 `base.html`에 노출되도록 다음과 같이 코드를 수정한다.

In [ ]:
<font color="red"><h1> My blog </h1></font>   <!-- base template을 분리해서 만든다 -->

{{username}} 님 로그인하셨습니다. <br><br><br>

{% block content %}
{% endblock %}

<br><br><br>
copyright....<br>
서울특별시...

## **부록) {% %}와 {{ }}의 차이**

### - {% %}는 커맨드다.
    - 얘를 만나면 django가 사전에 정의된 command를 실행한다.
        - {% for %}
        - {% if %}
        - {% url %}
        - {% block content %}
        - {% extends %}
        
### - {{ }}는 변수 호출용 플레이스홀더 같은 거다.
    - `render` 함수에 전달된 데이터에서 해당되는 값을 찾아서 그 자리에 끼워넣는다.
        - {{ d.title }}
            - `render`함수를 통해 템플릿에 전달된 d라는 키의 데이터에서 title을 찾아서 이 자리에 끼워넣는다.

# 4. 포스트 작성 폼 만들기
## 4.1 새로운 글 작성하기

 위에서 배운대로 `PostView`라는 클래스를 정의해서 get과 post를 개별 함수로 분리해서 처리하는 방법으로 구현했다.

먼저 `blog/urls.py`파일에는 이렇게 정의해준다.

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    ...
    path('add/', views.PostView.as_view(), name='add'),
    ...
]

`blog/views.py`에는 아래와 같이 써준다. 위에서 배운대로 class로 view를 정의해서 get과 post를 개별 함수로 분리해서 처리한다.

In [ ]:
class PostView(View):
    def get(self, request):
        return render(request, 'blog/edit.html')

    def post(self, request):
        title = request.POST.get('title')
        text = request.POST.get('text')
        username = request.session['username']
        user = User.objects.get(username=username)
        Post.objects.create(title=title, text=text, author=user)
        return redirect('list')

`get` 방식을 통한 요청에는 글 작성 폼을 띄워주고, `post` 방식으로 들어온 요청 내용으로 실제 글을 작성, 저장한 뒤에 글 목록으로 리다이렉트하는 형식이다.

`get`으로 들어온 요청으로 띄울 `blog.edit.html`은 다음과 같이 작성할 수 있다.

In [ ]:
<form action="{% url 'add' %}" method="post">
    {% csrf_token %}

    제목 <input type="text" name="title" /> <br>
    
    내용 <textarea name="text"> </textarea> <br>

    <input type="submit" value="작성" />

</form>

## 4.2 글 수정하기
`Form`을 활용해야 한다.

In [ ]:
from django.forms import Form
from django.forms import CharField, Textarea, ValidationError

In [ ]:
<to be continued>

### 4.2.1 Form을 쓰면 좋은점

- 여기저기서 호출해서 쓰기 편하다.
    - 별도의 객체로 만들 수 있기 때문에!
    - 특히 글을 수정하는 경우에 편리하게 사용할 수 있다; 기존 데이터 불러오기 & 저장하기
    
- 데이터를 통제할 수 있다
    - 특정 기준에 부합하지 않으면 에러를 발생시키도록 할 수 있기 때문에!